In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

def load_dataset(file_path):
    df = pd.read_csv(file_path, header=None, names=column_names, na_values='?')
    return df

cleveland = load_dataset("processed.cleveland.data")
hungary = load_dataset("processed.hungarian.data")
switzerland = load_dataset("processed.switzerland.data")
va = load_dataset("processed.va.data")

df = pd.concat([cleveland, hungary, switzerland, va], ignore_index=True)


In [10]:
df = df.dropna()

df['target'] = df['target'].apply(lambda x: 1 if int(x) > 0 else 0)


<ipython-input-10-d976dd131c62>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['target'].apply(lambda x: 1 if int(x) > 0 else 0)


In [11]:
categorical_cols = ['cp', 'restecg', 'slope', 'thal', 'ca', 'sex', 'fbs', 'exang']
df = pd.get_dummies(df, columns=categorical_cols)

X = df.drop("target", axis=1)
y = df["target"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [12]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.4008 - loss: 0.8131 - precision_2: 0.4481 - recall_2: 0.5504 - val_accuracy: 0.4167 - val_loss: 0.8119 - val_precision_2: 0.3235 - val_recall_2: 0.6875
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5602 - loss: 0.6778 - precision_2: 0.5437 - recall_2: 0.8194 - val_accuracy: 0.5833 - val_loss: 0.7215 - val_precision_2: 0.4333 - val_recall_2: 0.8125
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6852 - loss: 0.5921 - precision_2: 0.6247 - recall_2: 0.8727 - val_accuracy: 0.5833 - val_loss: 0.6532 - val_precision_2: 0.4286 - val_recall_2: 0.7500
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7715 - loss: 0.5228 - precision_2: 0.7426 - recall_2: 0.8323 - val_accuracy: 0.5833 - val_loss: 0.6045 - val_precision_2: 0.4286 - val_recall_2: 0.7500
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8587 - loss: 0.4309 - precision_2: 0.8503 - recall_2: 0.8895 - val_accuracy: 0.6458 - v

In [16]:
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8264 - loss: 0.3446 - precision_2: 0.7566 - recall_2: 0.8378
Accuracy: 0.83
Precision: 0.78
Recall: 0.84


In [14]:
print(df['target'].value_counts())


target
0    160
1    139
Name: count, dtype: int64




*   This is a relatively balanced dataset.

*   Class 0 (negative) makes up 53.5%, and class 1 (positive) makes up 46.5%.




Accuracy: 0.83 - 	83% of all predictions were correct — includes both positive and negative cases.

Precision: 0.78 - 	78% of predicted positive cases were correct (i.e., fewer false positives).

Recall: 0.84 - 	84% of actual positive cases were correctly detected (i.e., fewer false negatives).

Loss: 0.3446 - suggests stable learning and a good fit for binary classification.



*   model has improved in precision while maintaining strong recall.
*   Recall (0.84) is still higher than precision, which is common (and often acceptable) in medical predictions — it means your model is less likely to miss people who do have heart disease.




**MLP**


*   Application: Tabular/structured data (e.g., medical records, financial data).
*   Architecture: Fully connected layers with activation functions like ReLU and sigmoid.


*   Works well on structured datasets.
*   Easy to implement and train.


*   Cannot effectively capture spatial relationships (e.g., in image data).
*   Requires careful feature scaling and encoding.



**VS**






**CNN**


*   Application: Image and spatial data (e.g., object detection, handwriting recognition).
*   Architecture: Uses convolutional and pooling layers to learn spatial hierarchies.


*   Automatically extracts spatial features (edges, textures, shapes).
*   Outperforms MLPs on image tasks.


*   Requires more data and compute than MLPs.
*   Not suitable for non-spatial/tabular data.


---
**Effects of transfer learning**:
Using a CNN pretrained on a large dataset (e.g., ImageNet) and adapting it to a new task.

**How it works**:

*   Reuse early layers that capture general features.
*   Fine-tune later layers on your custom dataset.

Transfer learning significantly improves accuracy on small datasets, reduces training time and the risk of overfitting, and requires less labeled data.










